In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import argparse
import gensim

Using TensorFlow backend.


In [18]:
#Start of main script ....
skip_gram = 1
win_size = 20
dim = 50
len_seq = 200
perm = 'permutation_1_6_label'
#Load train/val/test subject list with labels
train_list = pd.read_csv('train_list_with_labels.csv', sep = '\t') #set_learning/
val_list = pd.read_csv('val_list_with_labels.csv', sep = '\t')  #set_learning/
test_list = pd.read_csv('test_list_with_labels.csv', sep = '\t') #set_learning/


#######################################
# Train w2v model under two scenarios #
#######################################
#with open("all_sequences_abnorm.txt", "r") as text_file:  #set_learning/
        #sequences = [i.replace('\n','') for i in text_file]   
#sequences = [i.split(' ') for i in sequences]
ori_sequences = pd.read_csv("all_train.csv")
perm = 'noperm'
if perm == 'noperm':
    sequences = [i.split(' ') for i in list(ori_sequences.seq)]
else: #othe perm only cases...
    perm_sequences = pd.read_csv(perm + ".csv")
    sequences = [i.split(' ') for i in list(perm_sequences.seq)]



In [19]:
######################################################
# Load train/val data and labels under two scenarios #
######################################################

#1) when no permutation
if perm == 'noperm':
	train_data = pd.read_csv('train_lab_abnorm_sc1.csv', sep = ',') #set_learning/
	#train_data = ori_sequences[ori_sequences.subject_id.isin(train_list.subject_id)] 
	val_data = pd.read_csv('val_lab_abnorm_sc1.csv', sep = ',') 

	#val_data = pd.read_csv('val_lab_abnorm.csv', sep = ',') #set_learning/
	test_data = pd.read_csv('test_lab_abnorm_sc1.csv', sep = ',')

	#train_y = list(train_list.HF)
	#val_y = list(val_list.HF)
	train_y = list(train_data.HF)#[int(train_list[train_list['subject_id']==i].HF) for i in train_data.subject_id]
	val_y = list(val_data.HF)#[int(val_list[val_list['subject_id']==i].HF) for i in val_data.subject_id]
	test_y = list(test_data.HF)#[int(test_list[test_list['subject_id']==i].HF) for i in test_data.subject_id]
    

#2) when permutation
elif perm == 'both':
    
	train_data_1 = pd.read_csv('train_lab_abnorm.csv', sep = ',')  #set_learning/
	val_data_1 = pd.read_csv('val_lab_abnorm.csv', sep = ',')#set_learning/
	
	train_y_1 = list(train_list.HF)
	val_y_1 = list(val_list.HF)

	train_data_2 = perm_sequences[perm_sequences.subject_id.isin(train_list.subject_id)]
	val_data_2 = perm_sequences[perm_sequences.subject_id.isin(val_list.subject_id)]
	del perm_sequences

	train_y_2 = [int(train_list[train_list['subject_id']==i].HF) for i in train_data_2.subject_id]
	val_y_2 = [int(val_list[val_list['subject_id']==i].HF) for i in val_data_2.subject_id]
	#combine the two parts
	
	train_data = pd.concat([train_data_1[['subject_id','seq']], train_data_2[['subject_id','seq']]])
	val_data = pd.concat([val_data_1[['subject_id','seq']], val_data_2[['subject_id','seq']]])

	train_y = train_y_1 + train_y_2
	val_y = val_y_1 + val_y_2

else: #other perm cases...

	perm_sequences = pd.read_csv(perm+".csv")
	train_data = perm_sequences[perm_sequences.subject_id.isin(train_list.subject_id)]
	#train_data = pd.read_csv('train_lab_abnorm.csv', sep = ',')
	val_data = perm_sequences[perm_sequences.subject_id.isin(val_list.subject_id)]
	test_data = ori_sequences[ori_sequences.subject_id.isin(test_list.subject_id)]

	del perm_sequences

	train_y = list(train_data.HF)#[int(train_list[train_list['subject_id']==i].HF) for i in train_data.subject_id]
	val_y = list(val_data.HF)#[int(val_list[val_list['subject_id']==i].HF) for i in val_data.subject_id]
	test_y = list(test_data.HF)#[int(test_list[test_list['subject_id']==i].HF) for i in test_data.subject_id]



In [20]:
def pad_trun_sequences(seq, len_seq):
    new_seq = list()
    for i in seq:
        #print(len(i))
        length= len(i)
        if length>len_seq:
             new_seq =  new_seq + [i[length-len_seq-1:-1]]
        if length<=len_seq:
             new_seq =  new_seq + [['00000']*(len_seq-length) +i]
                
    return new_seq          
            



train_x = pad_trun_sequences([i.split(' ') for i in train_data.seq][:train_data.count()[0]], len_seq )
#del train_data
val_x = pad_trun_sequences([i.split(' ') for i in val_data.seq][:val_data.count()[0]], len_seq )
#del val_data 
test_x = pad_trun_sequences([i.split(' ') for i in test_data.seq][:test_data.count()[0]], len_seq )
#del test_data
#[i.split(' ') for i in train_data.seq][0]

In [21]:
_model = gensim.models.Word2Vec(train_x+val_x+test_x, sg=skip_gram, window = win_size, iter=5, size= dim, min_count=1, workers=20)

In [22]:
embeddings_index = {}
embedding_matrix = np.zeros((len(_model.wv.vocab) + 1, dim))

for i, word in enumerate(_model.wv.vocab):
    coefs = np.asarray(_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs
    embedding_matrix[i] = coefs
print('Found %s word vectors.' % len(embeddings_index))


Found 188 word vectors.


In [23]:
embedding_matrix

array([[ 0.37235039, -0.17826508, -0.46533552, ...,  0.84098381,
         0.15317829,  0.14197485],
       [ 0.2335694 , -0.26351935,  0.06532324, ...,  0.26811111,
        -0.05035434,  0.09372307],
       [ 0.22731383, -0.24896312,  0.07727272, ...,  0.43054605,
         0.02942917,  0.43146661],
       ...,
       [-0.1878562 ,  0.29569817, -0.10441412, ...,  0.0747788 ,
        -0.28476393, -0.03737698],
       [ 0.13212186, -0.07086154,  0.09452537, ...,  0.03970203,
        -0.40025756, -0.33719414],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [24]:
word_index = {}
for i, word in enumerate(_model.wv.vocab):
    word_index[word] = i

In [25]:
def encoder(seq):
    seq_encoded = list()
    for i in seq:
        seq_encoded = seq_encoded + [[word_index[j] for j in i]]       
    return seq_encoded

In [26]:
train_x_encoded = encoder(train_x)
val_x_encoded = encoder(val_x)
test_x_encoded = encoder(test_x)

In [27]:
import tensorflow as tf

In [28]:
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.layers import LSTM,GRU,Conv1D,MaxPooling1D,Flatten,Embedding,LeakyReLU
from keras.layers.embeddings import Embedding
import keras_metrics
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import TensorBoard

ksize = 3


sequence_input = Input(shape=(len_seq,), dtype='int32')
embedded_sequences = Embedding(len(embeddings_index) + 1, dim,
                            weights=[embedding_matrix],
                            input_length=len_seq,
                            trainable=False)(sequence_input)
x = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid', dilation_rate=1,  activation=None, use_bias=True)(embedded_sequences)
x = LeakyReLU(alpha=0.3)(x)
x = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid',dilation_rate=1,  activation=None, use_bias=True)(x)
x = LeakyReLU(alpha=0.3)(x)
x = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid', dilation_rate=1, activation=None, use_bias=True)(x)
x = LeakyReLU(alpha=0.3)(x)  # global max pooling
x = GRU(cnn_dim,dropout = 0.2)(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(sequence_input, preds)

#model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

# happy learning!
model.fit(np.array(train_x_encoded),train_y, validation_data=(np.array(val_x_encoded), val_y),
          epochs=10, batch_size=128)

Train on 5426 samples, validate on 904 samples
Epoch 1/10
5426/5426 [==============================] - 2s 424us/step - loss: 0.6208 - acc: 0.6587 - val_loss: 0.5954 - val_acc: 0.6748
Epoch 2/10
5426/5426 [==============================] - 2s 281us/step - loss: 0.5853 - acc: 0.6756 - val_loss: 0.5777 - val_acc: 0.6991
Epoch 3/10
5426/5426 [==============================] - 2s 283us/step - loss: 0.5750 - acc: 0.6924 - val_loss: 0.5721 - val_acc: 0.6858
Epoch 4/10
5426/5426 [==============================] - 2s 286us/step - loss: 0.5649 - acc: 0.6976 - val_loss: 0.5707 - val_acc: 0.6936
Epoch 5/10
5426/5426 [==============================] - 2s 304us/step - loss: 0.5511 - acc: 0.7145 - val_loss: 0.5742 - val_acc: 0.7069
Epoch 6/10
5426/5426 [==============================] - 2s 288us/step - loss: 0.5324 - acc: 0.7250 - val_loss: 0.5912 - val_acc: 0.6969
Epoch 7/10
5426/5426 [==============================] - 2s 283us/step - loss: 0.5214 - acc: 0.7396 - val_loss: 0.6116 - val_acc: 0.7179
E

In [155]:
np.array(train_x_encoded).shape

(5426, 200)

In [146]:
embedding_layer

In [ ]:
model = Sequential()
#sequence_input = Input(shape=(len_seq,), dtype='ufloat32')
#embedded_sequences = embedding_layer(sequence_input)
model.add(Embedding(len(embeddings_index) + 1, dim,
                            weights=[embedding_matrix],
                            input_length=len_seq,
                            trainable=False))
 ksize=3
    i = Input(batch_shape=(batch_size, timesteps, input_dim))
    o = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid', dilation_rate=1,  activation=None, use_bias=True)(i)
    o = LeakyReLU(alpha=0.3)(o)
    o = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid',dilation_rate=1,  activation=None, use_bias=True)(i)
    o = LeakyReLU(alpha=0.3)(o)
    o = Conv1D(cnn_dim, kernel_size=ksize, strides=ksize, padding='valid', dilation_rate=1, activation=None, use_bias=True)(i)
    o = LeakyReLU(alpha=0.3)(o)
    o = GRU(cnn_dim,dropout = 0.2)(o)
    o = Dense(1,activation='sigmoid')(o)

    m = Model(inputs=[i], outputs=[o])